In [2]:
# Esperimenti sull'output di inferenza a mano
import tensorflow as tf
import numpy as np
from PIL import Image

nomeModello = "Modelli/Utili/yolo11n_float32.tflite"
nomeImmagine = "Foto/example4_preprocessed.jpg"

#converte l'immagine in RGB
img = Image.open(nomeImmagine).convert('RGB')

# Converti in un tensore 3d con valori float32 e normalizza (valori da 0 a 1)
input_data =  np.array(img).astype(np.float32) / 255.0

#aggiungi dimensione batch (1, 640, 640, 3) (Yolo si aspetta sempre un batch di input)
#YOLO richiede input di forma (batch_size, height, width, channels) (ottenendo un tensore 4d)
input_data = np.expand_dims(input_data, axis=0)

#carica il modello
interpreter = tf.lite.Interpreter(model_path=nomeModello)
interpreter.allocate_tensors() #alloca memoria per i tensori

#formati input/output modello
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("formato dell'input del modello:", input_details[0]['dtype'])
print("formato output del modello:", output_details[0]['dtype'])

# --- Imposta tensore input ---
interpreter.set_tensor(input_details[0]['index'], input_data) 

# --- Esegui inferenza ---
interpreter.invoke()

# --- Ottieni output ---
output_data = interpreter.get_tensor(output_details[0]['index'])
predictions = output_data[0]

# --- Analisi output ---
print("Shape output:", output_data.shape)
predictions = predictions.T #faccio la trasposta per avere su ogni riga una predizione
print("predictions shape:", predictions.shape)
#print("predictions:", predictions)
print("prima riga:", predictions[1002])

objectness = predictions[:, 4]
class_0_probs = predictions[:, 5]







formato dell'input del modello: <class 'numpy.float32'>
formato output del modello: <class 'numpy.float32'>
Shape output: (1, 84, 8400)
predictions shape: (8400, 84)
prima riga: [5.40647328e-01 1.51085123e-01 8.51498842e-02 4.00711894e-02
 2.79087430e-07 6.26703880e-08 3.29604177e-07 6.01762480e-08
 1.32663345e-07 7.62738921e-08 7.28267437e-08 6.29471231e-08
 1.61372540e-07 6.91916711e-08 1.55344235e-08 6.17176070e-08
 2.56964405e-08 1.74080597e-07 2.06534068e-07 6.85886832e-08
 5.10500868e-08 7.56553575e-08 5.57399922e-08 8.17181132e-08
 1.03012141e-07 6.16727718e-08 7.68923556e-08 7.43975193e-08
 1.57854444e-07 3.22302526e-07 1.44844464e-07 1.74556220e-07
 7.69884778e-08 1.58081775e-07 2.65187907e-07 9.33271380e-08
 7.09081931e-08 1.48563984e-07 2.10989526e-07 4.97726766e-08
 6.40138040e-08 2.40848721e-07 8.98006363e-08 4.40786572e-08
 3.20233156e-08 6.13186657e-08 2.64282818e-07 2.86992559e-07
 2.62695181e-07 1.63428197e-07 2.01161612e-07 1.06888720e-07
 4.81617128e-08 8.73457680e-0

In [ ]:
#Usiamo la libreria ultralytics per fare inferenza con YOLO11n
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt") #esporta in .pt

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [ ]:
model.export(format="onnx") #esporta in .onnx

In [ ]:
# Inference and save results with Ultralytics
nomeImmagine = "../../Calibrazione/immagini_calibrazione640x480/capture(1).jpg"
results = model.predict(
    nomeImmagine, 
    save=True, 
    project="../../FotoInference",
    name="detection",
)
print("results:", results)

In [ ]:
#quantizzazione del modello int int8 .espdl
import sys, os
import esp_ppq
sys.path.append(os.path.abspath("esp-detection"))
from deploy.quantize import quant_espdet

quant_espdet(
    onnx_path="yolo11n.onnx",      # onnx appena esportato
    target="esp32s3",              
    num_of_bits=8,                 # quantizzazione int8
    device='cpu',
    batchsz=1,
    imgsz=[640, 640],              # deve combaciare con la dimensione del modello
    calib_dir="../../calibrazione_espressif/",        #  cartella immagini reali
    espdl_model_path="yolo11n.espdl"
)


In [ ]:
#export ONNX di espressif

from ultralytics import YOLO
from ultralytics.nn.modules import Detect, Attention
from ultralytics.engine.exporter import Exporter, try_export, arange_patch
from ultralytics.utils import LOGGER, __version__, colorstr
from ultralytics.utils.checks import check_requirements
from ultralytics.utils.torch_utils import get_latest_opset
import torch
import onnx


class ESP_Detect(Detect):
    def forward(self, x):
        """Returns predicted bounding boxes and class probabilities respectively."""
        # self.nl = 3
        box0 = self.cv2[0](x[0])
        score0 = self.cv3[0](x[0])

        box1 = self.cv2[1](x[1])
        score1 = self.cv3[1](x[1])

        box2 = self.cv2[2](x[2])
        score2 = self.cv3[2](x[2])

        return box0, score0, box1, score1, box2, score2


class ESP_Attention(Attention):
    def forward(self, x):
        """
        Forward pass of the Attention module.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            (torch.Tensor): The output tensor after self-attention.
        """
        B, C, H, W = x.shape
        N = H * W
        qkv = self.qkv(x)
        q, k, v = qkv.view(
            -1, self.num_heads, self.key_dim * 2 + self.head_dim, N
        ).split([self.key_dim, self.key_dim, self.head_dim], dim=2)
        attn = (q.transpose(-2, -1) @ k) * self.scale
        attn = attn.softmax(dim=-1)
        x = (v @ attn.transpose(-2, -1)).view(-1, C, H, W) + self.pe(
            v.reshape(-1, C, H, W)
        )
        x = self.proj(x)
        return x


class ESP_Detect_Exporter(Exporter):
    """
    adapted from ultralytics for detection task
    """

    @try_export
    def export_onnx(self, prefix=colorstr("ONNX:")):
        """YOLO ONNX export."""
        requirements = ["onnx>=1.14.0"]  # from esp-ppq requirments.txt
        # since onnxslim will cause NCHW -> 1(N*C)HW in yolo11, we replace onnxslim with onnxsim
        if self.args.simplify:
            requirements += [
                "onnxsim",
                "onnxruntime" + ("-gpu" if torch.cuda.is_available() else ""),
            ]
        check_requirements(requirements)

        opset_version = self.args.opset or get_latest_opset()
        LOGGER.info(
            f"\n{prefix} starting export with onnx {onnx.__version__} opset {opset_version}..."
        )
        f = str(self.file.with_suffix(".onnx"))
        output_names = ["box0", "score0", "box1", "score1", "box2", "score2"]
        dynamic = (
            self.args.dynamic
        )  # case 1: deploy model on ESP32, dynamic=False; case 2: QAT gt onnx for inference, dynamic=True
        if dynamic:
            dynamic = {"images": {0: "batch"}}
            for name in output_names:
                dynamic[name] = {0: "batch"}

        with arange_patch(self.args):
            torch.onnx.export(
                self.model,
                self.im,
                f,
                verbose=False,
                opset_version=opset_version,
                do_constant_folding=False,
                input_names=["images"],
                output_names=output_names,
                dynamic_axes=dynamic or None,
            )
        # Checks
        model_onnx = onnx.load(f)  # load onnx model

        # Simplify
        if self.args.simplify:
            try:
                import onnxsim

                LOGGER.info(
                    f"{prefix} simplifying with onnxsim {onnxsim.__version__}..."
                )
                model_onnx, _ = onnxsim.simplify(model_onnx)

            except Exception as e:
                LOGGER.warning(f"{prefix} simplifier failure: {e}")

        # Metadata
        for k, v in self.metadata.items():
            meta = model_onnx.metadata_props.add()
            meta.key, meta.value = k, str(v)

        onnx.save(model_onnx, f)
        return f, model_onnx


class ESP_YOLO(YOLO):
    def export(
        self,
        **kwargs,
    ):
        self._check_is_pytorch_model()
        custom = {
            "imgsz": self.model.args["imgsz"],
            "batch": 1,
            "data": None,
            "device": None,
            "verbose": False,
        }
        args = {**self.overrides, **custom, **kwargs, "mode": "export"}
        return ESP_Detect_Exporter(overrides=args, _callbacks=self.callbacks)(
            model=self.model
        )


model = ESP_YOLO("yolo11n.pt")
for m in model.modules():
    if isinstance(m, Attention):
        m.forward = ESP_Attention.forward.__get__(m)
    if isinstance(m, Detect):
        m.forward = ESP_Detect.forward.__get__(m)

model.export(format="onnx", simplify=True, opset=13, dynamic=False, imgsz=640)